In [140]:
import agentpy as ap
import matplotlib.pyplot as plt
import IPython

In [141]:
# Road agent positions

#ROUNDABOUT_X = []
#ROUNDABOUT_Y = []

ROUNDABOUT = []
POSITIONS_X = []
POSITIONS_Y = []

'''
for i in range(0, 26):
    POSITIONS_X.append((13,i))
    POSITIONS_Y.append((i,13))
'''
for i in range(0, 11):
    POSITIONS_X.append((13,i))
for i in range(16, 27):
    POSITIONS_X.append((13,i))

for i in range(17, 27):
    POSITIONS_Y.append((i,13))
for i in range(0,11):
    print(0)
    #POSITIONS_Y.append((i,13))

def midPointCircleDraw(x_centre, y_centre, ratio):
        x = ratio
        y = 0
        
        # Printing the initial point the
        # axes after translation
        ROUNDABOUT.append((x + x_centre, y + y_centre))
        print("(", x + x_centre, ", ",
                  y + y_centre, ")",
                  sep = "", end = "")
        
        # When radius is zero only a single
        # point be printed
        '''
        if (ratio > 0) :
        
            print("(", x + x_centre, ", ",
                      -y + y_centre, ")",
                      sep = "", end = "")
            ROUNDABOUT.append((x + x_centre, -y + y_centre))
            print("(", y + x_centre, ", ",
                      x + y_centre, ")",
                      sep = "", end = "")
            ROUNDABOUT.append((y + x_centre, x + y_centre))
            print("(", -y + x_centre, ", ",
                        x + y_centre, ")", sep = "")
            ROUNDABOUT.append((-y + x_centre, x + y_centre))
        '''
        # Initializing the value of P
        P = 1 - ratio
        print(P)
 
        while x > y:
        
            y += 1
            
            # Mid-point inside or on the perimeter
            if P <= 0:
                P = P + 2 * y + 1
                
                
            # Mid-point outside the perimeter
            else:        
                x -= 1
                P = P + 2 * y - 2 * x + 1
            
            # All the perimeter points have
            # already been printed
            if (x < y):
                break
            
            # Printing the generated point its reflection
            # in the other octants after translation
        
            #ROUNDABOUT.append((x + x_centre, y + y_centre))
            ROUNDABOUT.append((-x + x_centre, y + y_centre))
            #ROUNDABOUT.append((x + x_centre, -y + y_centre))
            ROUNDABOUT.append((-x + x_centre, -y + y_centre))
            
            # If the generated point on the line x = y then
            # the perimeter points have already been printed
            
            if x != y:
                print("(", y + x_centre, ", ",
                      x + y_centre, ")",
                      sep = "", end = "")
                #ROUNDABOUT.append((y + x_centre, x + y_centre))

                print("(", -y + x_centre, ", ",
                      x + y_centre, ")",
                      sep = "", end = "")
                ROUNDABOUT.append((-y + x_centre, x + y_centre))

                print("(", y + x_centre, ", ",
                      -x + y_centre, ")",
                      sep = "", end = "")
                #ROUNDABOUT.append((y + x_centre, -x + y_centre))

                print("(", -y + x_centre, ", ",
                      -x + y_centre, ")",
                      sep = "", end = "")
                ROUNDABOUT.append((-y + x_centre, -x + y_centre))
                
                             
# Driver Code
#if __name__ == '__main__':
        
  # To draw a circle of radius 3
  # centered at (0, 0)

midPointCircleDraw(13, 13, 3)
ROUNDABOUT.append((10,13))


0
0
0
0
0
0
0
0
0
0
0
(16, 13)-2
(14, 16)(12, 16)(14, 10)(12, 10)

In [142]:
class CleaningRoomsModel(ap.Model):
    def setup(self):
        
        # Create agents
    
        self.cars=ap.AgentList(self,2)
        road=ap.AgentList(self,len(ROUNDABOUT))
        roadX=ap.AgentList(self,len(POSITIONS_X))
        roadY=ap.AgentList(self,len(POSITIONS_Y))
        self.sign=ap.AgentList(self,2)

        # Create grid
        self.city = ap.Grid(self, [self.p.size] * 2, track_empty=True)
        
        self.city.add_agents(self.cars, [(26,13),(13,26)])
        self.city.add_agents(road, positions=ROUNDABOUT ,empty=False)
        self.city.add_agents(roadX, positions=POSITIONS_X ,empty=False)
        self.city.add_agents(roadY, positions=POSITIONS_Y, empty=False)
        self.city.add_agents(self.sign, [(5,5),(5,20)])

        # Agent type attribute
        
        #0: car 0
        #1: untravelled road
        #2: travelled road
        #7: car 1

        self.cars.type_agent = 0
        road.type_agent = 1
        roadX.type_agent = 1
        roadY.type_agent = 1

        #3: green light
        #4: red light
        self.sign[0].type_agent = 3
        self.sign[1].type_agent = 4

        #0: vertical
        #1: horizontal 
        self.cars[0].type_car = 0
        self.cars[1].type_car = 1
        
        self.cars[0].type_agent = 0
        self.cars[1].type_agent = 7

        # Road direction attribute
        #5: is horizontal
        #6: is vertical
        road.road_direction = 5
        roadX.road_direction = 5
        roadY.road_direction = 6
        
        road.road_type = 1
        roadX.road_type = 2
        roadY.road_type = 3

    def step(self):
        cars = self.cars
        sigb = self.sign[0]
        sigc = self.sign[1]
        
        if sigb.type_agent == 3:
            sigb.type_agent = 4

        elif sigb.type_agent == 4:
            sigb.type_agent = 3
            
        if sigc.type_agent == 3:
            sigc.type_agent = 4

        elif sigc.type_agent == 4:
            sigc.type_agent = 3


        for car in cars:
            old_position=self.city.positions[car]
            
            for neighbor in self.city.neighbors(car):
                # num_neighbors = self.city.neighbors(car).size()

                new_position = self.city.positions[neighbor]
                
                '''
                if neighbor.type_agent == 1 and sigb.type_agent == 3 and neighbor.road_direction == 6 and car.type_car == 0:
                    self.city.move_to(car, new_position)
                    neighbor.type_agent = 2
                    break

                if neighbor.type_agent == 1 and sigc.type_agent == 3 and neighbor.road_direction == 5 and car.type_car == 1:
                    self.city.move_to(car, new_position)
                    neighbor.type_agent = 2                    
                    break
                '''
                
                if neighbor.type_agent == 1 and sigb.type_agent == 3 and car.type_car == 0:
                    self.city.move_to(car, new_position)
                    neighbor.type_agent = 2
                    break

                if neighbor.type_agent == 1 and sigc.type_agent == 3 and car.type_car == 1:
                    self.city.move_to(car, new_position)
                    neighbor.type_agent = 2                    
                    break

'''
        for car in cars:
            old_position=self.city.positions[car]
            pos_neighbors = []
            for neighbor in self.city.neighbors(car):
                new_position = self.city.positions[neighbor]
                pos_neighbors.append(new_position)
            
            for pos in pos_neighbors:
                #si el carro está abajo y entra a la glorieta gira a la derecha
                if pos[0] >= 13 and pos[1] > 13:
                  self.city.move_to(car, pos)
                  break
                #elif pos[0] >= 13 and pos[1]


                #guardar las posiciones de sus vecinos en un vector
                #checar de esas posiciones cuál es la más óptima y moverse a esa

                print("positions")
                print(self.city.positions[neighbor])
                #if(self.city.positions[neighbor]) 

                if neighbor.type_agent == 1 and sigb.type_agent == 3 and car.type_car == 0:
                    if neighbor.road_type == 3:
                      self.city.move_to(car, new_position)
                      neighbor.type_agent = 2 
                    elif neighbor.road_type == 1:
                      self.city.move_to(car, new_position)
                      neighbor.type_agent = 2 
                    else:
                      self.city.move_to(car, new_position)
                      neighbor.type_agent = 2 
                    break

'''

                
'''
                if neighbor.type_agent == 1 and sigc.type_agent == 3 and car.type_car == 1:

                    if neighbor.road_type == 3:
                      self.city.move_to(car, new_position)
                      neighbor.type_agent = 2 
                    elif neighbor.road_type == 2:
                      self.city.move_to(car, new_position)
                      neighbor.type_agent = 2 
                    else:
                      self.city.move_to(car, new_position)
                      neighbor.type_agent = 2 
                    break
                     
'''
                
                


'\n                if neighbor.type_agent == 1 and sigc.type_agent == 3 and car.type_car == 1:\n\n                    if neighbor.road_type == 3:\n                      self.city.move_to(car, new_position)\n                      neighbor.type_agent = 2 \n                    elif neighbor.road_type == 2:\n                      self.city.move_to(car, new_position)\n                      neighbor.type_agent = 2 \n                    else:\n                      self.city.move_to(car, new_position)\n                      neighbor.type_agent = 2 \n                    break\n                     \n'

In [143]:
# Define parameters
parameters={
    'size':27,
    'steps':50,
}
# Create single-run animation with custom colors
def animation_plot (model, ax):
    attr_grid=model.city.attr_grid('type_agent')
    color_dict = {0:'#ff00ea', 1:'#808080', 2: '#808080',3:'#15b032',4:'#eb361a', 7:'#0062ff', None: '#497a41'}
    ap.gridplot (attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Traffic simulation\n"
    f"Time-step: {model.t}")
fig, ax = plt. subplots()
model=CleaningRoomsModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))